In [ ]:
import pandas as pd
import os
import sys
import numpy as np
sys.path.append('..')
from tqdm import tqdm
from soundbay.utils.metadata_processing import bg_from_non_overlap_calls

In [ ]:
def process_gzip_files(input_dir):
    # find all .txt files in the dir
    all_files = os.listdir(input_dir)
    txt_files = [f for f in all_files if f.endswith('.txt')]
    print(f'Found {len(txt_files)} txt files')
    for txt_file in txt_files:
        txt_file_path = os.path.join(input_dir, txt_file)
        print(f'Processing {txt_file_path}')
        # Rename the txt to include .gz at the end. For example mv tomer.txt tomer.txt.gz
        # unzip using gzip: gzip -d tomer.txt.gz
        os.system(f'mv "{txt_file_path}" "{txt_file_path}.gz"')
        os.system(f'gzip -d "{txt_file_path}.gz"')





In [ ]:
# file_path = '~/Downloads/FlumeL_m_23.04.11.Sounds.selections_DV (1).txt'
def get_files(main_dir):

    files = []
    for r, d, f in os.walk(main_dir):
        for file in f:
            files.append(os.path.join(r, file))
    files = [f for f in files if f.endswith('.txt')]
    all_dfs = {}
    for file in files:
        df = pd.read_csv(file, sep='\t')
        # leave filename basename
        file = os.path.basename(file)
        all_dfs[file] = df
    return all_dfs

In [ ]:
    # df = pd.read_csv(file_path, sep="\t")
# main_dir = '/Users/tomernahshon/Documents/soundbay/shaye_data/all_txt_9.3.24_concluded'
main_dir = '/datadrive/soundbay_backup/txt_files_19.10.24/'
# process_gzip_files(main_dir)
all_dfs = get_files(main_dir)
# df.head()


In [ ]:
all_dfs = get_files(main_dir)

In [ ]:
all_dfs

In [ ]:
for i, (filename, df) in enumerate(all_dfs.items()):
    # strip filename to file name only
    # filename = filename.split('/')[-1]
    if 'sounds' in filename.lower():
        print(f'file: {filename} , shape: {df.shape}')


In [ ]:
def process_df(df):
    # df_copy = df.copy()
    # df['s3_filepath'] = file_path
    df['filename'] = df['Begin File'][:-4]
    df['call_length'] = df['End Time (s)'] - df['Begin Time (s)']
    df['begin_time'] = df['File Offset (s)']
    df['end_time'] = df['begin_time'] + df['call_length']
    df['label'] = 1
    return df

In [ ]:
# all_dfs['FlumeL_m_23.04.11.Sounds.selections_DV.txt'].head()

# processed_df = process_df(all_dfs['FlumeL_m_23.04.11.Sounds.selections_DV.txt'])

# remove all columns that are not part of process_df function
# columns_to_keep = ['filename', 'call_length', 'begin_time', 'end_time', 'label']
# remove all other columns
# processed_df = processed_df[columns_to_keep]


# Begin File - file name
# Begin path - file path (change to s3 folder bucket)
# call_length - all_dfs['End Time (s)'] - all_dfs['FlumeL_m_23.04.11.Sounds.selections_DV.txt']['Begin Time (s)']
# label
# filename
# begin_time - File Offset (s)
# end_time - begin_time + call_length

In [ ]:
def get_folders(s3_path, my_bucket='deepvoice-user-uploads'):
    import boto3
    list_of_folders = []
    client = boto3.client('s3')
    result = client.list_objects(Bucket=my_bucket, Prefix=s3_path, Delimiter='/')
    for o in result.get('CommonPrefixes'):
            fold = o.get('Prefix').strip(s3_path)
            if 'wav' in fold:
                    list_of_folders.append(fold)


    return list_of_folders

In [ ]:
def check_file_exists(s3_path, filename, s3, folders_list, my_bucket='deepvoice-user-uploads'):
    # folders_list = ['FlumeL_m_23.03.26-23.04.11_wav shaye tudor'
    #             ,'FlumeL_m_23.04.11-23.04.23_wav shaye tudor'
    #             ,'FlumeL_m_23.04.24-23.05.09_wav shaye tudor'
    #             ,'LOOT_03.10.23_wav shaye tudor'
    #             ,'LOOT_23.03.10-23.03.28_wav shaye tudor'
    #             ,'LOOT_23.03.29-23.04.12_wav shaye tudor']
    for folder in folders_list:

        try:
            path = os.path.join(s3_path , folder , filename)
            # print(path)
            s3.head_object(Bucket=my_bucket, Key=path)
            # print(f"The file {filename} exists in the S3 path.")
            return 's3://' + my_bucket + '/' + s3_path + folder
        except:
            # print(f"The file {filename} does not exist in the S3 path.")
            pass
        
    return None



In [ ]:
s3_parent_folder = 'shayetudor@gmail.com/dropbox/cods/'
list_of_folders = [i[:-1] + ' tudor' for i in get_folders(s3_parent_folder)]

In [ ]:
list_of_folders

In [ ]:
columns_to_keep = ['filename', 'call_length', 'begin_time', 'end_time', 'label']
import boto3
AWS_ACCESS_KEY = ''
AWS_SECRET_KEY = ''
# s3://deepvoice-user-uploads
s3_parent_folder = 'shayetudor@gmail.com/dropbox/cods/'

# filename = '7205.230413160010.wav'
# Check if filename exists in s3_path using boto3 head function
# s3 = boto3.client('s3', aws_access_key_id=AWS_ACCESS_KEY, aws_secret_access_key=AWS_SECRET_KEY)
s3 = boto3.client('s3')



all_edited_dfs = []
for j, (annotations_filename, df) in enumerate(tqdm(all_dfs.items())):

    if 'sounds' in annotations_filename.lower():
        processed_df = process_df(df)

        # remove all other columns
        processed_df = processed_df[columns_to_keep]
        processed_df['annotations_filename'] = annotations_filename
        for i, row in processed_df.iterrows():
            if type(row['filename']) != str:
                continue
            filename = row['filename']
            s3_specific_path = check_file_exists(s3_parent_folder, filename, s3, list_of_folders)
            if s3_specific_path:
                processed_df.loc[i, 's3_path'] = s3_specific_path
            else:
                processed_df.loc[i, 's3_path'] = 'None'
        all_edited_dfs.append(processed_df)


all_edited_dfs = pd.concat(all_edited_dfs)

        

In [ ]:
all_edited_dfs['annotations_filename'].shape

In [ ]:
all_edited_dfs.head()

In [ ]:
all_edited_dfs = all_edited_dfs[all_edited_dfs['annotations_filename'] != 'FlumeL_m_sample.quiet_Sounds.selections.txt']

In [ ]:
all_edited_dfs.head()

In [ ]:
all_edited_dfs = all_edited_dfs[(all_edited_dfs['s3_path'] != 'None')].dropna()


In [ ]:
all_edited_dfs.shape

In [ ]:
all_edited_dfs['call_length'].round(2).value_counts()

In [ ]:
# Remove heavy outliers
all_edited_dfs = all_edited_dfs[all_edited_dfs['call_length'] < 10]

In [ ]:
b = all_edited_dfs.sort_values('begin_time', ascending=True)

In [ ]:
newdf = bg_from_non_overlap_calls(b)

In [ ]:
# newdf[newdf['label'] == 0].head()

In [ ]:
newdf.columns

In [ ]:
row = newdf.iloc[200]

In [ ]:
row

In [ ]:
newdf['s3_path'].iloc[0]

In [ ]:
# df['col_3'] = df.apply(lambda x: f(x.col_1, x.col_2), axis=1)

newdf['filename'] = newdf['filename'].apply(lambda x: str(x[:-4]))
# newdf['s3_path'] = newdf['s3_path'].apply(lambda x: x.split('/')[-1])

In [ ]:
newdf['s3_path'].iloc[3]

In [ ]:
newdf['s3_path'] = newdf['s3_path'] + '/' + newdf['filename'] + '.wav'

In [ ]:
newdf['s3_path'].iloc[100]

In [ ]:
newdf.to_csv('/datadrive/soundbay_backup/soundbay/shaye_annotations_22_10_24.csv', index=False)